In [1]:
#import packages
import pandas as pd
import requests
import os as os
import json 
import tweepy
from timeit import default_timer as timer
import ast

# Gathring #

In [2]:
df_csv=pd.read_csv("twitter-archive-enhanced.csv")

In [3]:
imgpre_url="https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"

In [4]:
imgpre_filename=imgpre_url.split("/")[-1]

In [5]:
response=requests.get(imgpre_url)

In [6]:
with open(imgpre_filename,mode="wb") as file:
    file.write(response.content)

In [7]:
df_img=pd.read_csv(imgpre_filename,sep="\t")

## Prapring the api creditial from json file ##

#you need to write your api in the attached json file"
credintials=pd.read_json("TweeterApiToken.json")
credintials.head()

In [9]:
consumer_key = credintials.tokens.APIkey
consumer_secret = credintials.tokens.APISecretkey
access_token = credintials.tokens.AccessToken
access_secret = credintials.tokens.AccessTokenSecret

In [10]:
auth=tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_secret)

In [11]:
api=tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [12]:
# you need to change 0 to 1 if you want to start reading the tweets 
if (0) :
    start = timer()
    failer={}
    Readinglog=""
    n=0
    with open ("tweet_json.txt",mode="w") as file:
        for tweetid in df_csv.tweet_id:
            n=n+1
            Readinglog+=(str(n)+" "+str(tweetid)+" ")
            print ( n," : ",timer() )
            try:
                temp=api.get_status(tweetid, tweet_mode='extended')._json
                Readinglog+="Success "
                json.dump(temp,file)            
                file.writelines("\n")
            except tweepy.TweepError as e:
                Readinglog+="Faild "
                failer[tweetid]=e
                pass
            #print(temp["favorite"])#favorite
            Readinglog+="\n"
            if n == 10000:
                break
    end = timer()
    print ( end - start)
    
    start = timer()
    failer={}
    Readinglog1=""
    n=0
    with open ("tweet_json1.txt",mode="w") as file:
        for tweetid in df_csv.tweet_id:
            n=n+1
            Readinglog1+=(str(n)+" "+str(tweetid)+" ")
            print ( n," : ",timer() )
            try:
                temp=api.get_status(tweetid, tweet_mode='extended')._json
                Readinglog1+="Success "
                json.dump(temp,file)            
                file.writelines("\n")
            except tweepy.TweepError as e:
                Readinglog1+="Faild "
                failer[tweetid]=e
                pass
            #print(temp["favorite"])#favorite
            Readinglog1+="\n"
            if n == 10:
                break

    end = timer()
    print ( end - start)

In [13]:
#https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html
with open ("tweet_json1.txt",mode="r") as file:
    lines=file.readlines()

json.loads(lines[3])

In [15]:
with open ("tweet_json.txt",mode="r") as file:
    lines=file.readlines()

In [16]:
df_list=[]
for line in lines:
    line=json.loads(line)
    df_list.append({
        "tweet_id":line["id"],
        "favorite_count":line["favorite_count"],
        "retweet_count":line["retweet_count"],
           })

In [17]:
df_api=pd.DataFrame(df_list)

df_api.head()

df_img.head()

df_csv.head(50)

 # Assest  # 
## Quality: ## 
 
 ### df_csv ###
 1 - time stamp need to be changed to datatime instead of string<br/>
 2 - dummy varialbs ( dog type ) need to be cleaned into int ( boolean)<br/>
 3 - in_reply_to_status_id should be changed to string type <br/>
 4 - retweeted_status_user_id  should be changed to string type <br/>
 5 - rating_numerator ,rating_denominator  changing into one float variable<br/>
 <br/>
 ### df_img: ###
 changing id's into string <br/>
 <br/>
 #### df_api ####
 <br/>
 changing id's into string
 
 

df_csv.head()

df_csv.info()

In [23]:
df_csv.doggo.unique()

array(['None', 'doggo'], dtype=object)

In [24]:
df_csv.floofer.unique()

array(['None', 'floofer'], dtype=object)

In [25]:
df_csv.puppo.unique()

array(['None', 'puppo'], dtype=object)

In [26]:
df_csv.pupper.unique()

array(['None', 'pupper'], dtype=object)

df_csv[df_csv.expanded_urls.isna()].info()

df_csv[df_csv.expanded_urls.isna()].head()

In [29]:
df_csv[df_csv.tweet_id.duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


df_img.head()

df_img.info()

df_img.p3.unique()

In [33]:
df_img.img_num.unique()

array([1, 4, 2, 3], dtype=int64)

df_img[df_img.img_num==4].info()

df_img[df_img.img_num==4].head()

 df_api.head()

df_api.info()

  # Assest  # 

 ## tideness ##
 
1 -  df_api , df_csv  represent the same obeservations it would better to be merged together
 but we should be careful about the nullable and deference count between the raw<br/>
2 - merged , df_img represent the same obeservations it would better to be merged together
 but we should be careful about the nullable and deference count between the raw

# Cleaning #
## Quality ##

### df_csv ###
1- chaging time into datatime type

In [38]:
df_csv["timestamp"]=pd.to_datetime(df_csv.timestamp)

df_csv.info()

### df_csv ###
2- chaging retweeted_status_id and other id's into string type


In [40]:
df_csv["tweet_id"]=df_csv["tweet_id"].astype(str)
df_csv["retweeted_status_id"]=df_csv["retweeted_status_id"].astype(str)
df_csv["retweeted_status_user_id"]=df_csv["retweeted_status_user_id"].astype(str)
df_csv["in_reply_to_status_id"]=df_csv["in_reply_to_status_id"].astype(str)
df_csv["in_reply_to_user_id"]=df_csv["in_reply_to_user_id"].astype(str)


### df_csv ###
3- changing rating into one column and droping the numerator and denominator note ( denominator = 0 ) should be droped


In [41]:
df_csv=df_csv[df_csv["rating_denominator"]!=0]

In [42]:
df_csv["rating"]=df_csv.rating_numerator / df_csv.rating_denominator

In [43]:
df_csv.drop(["rating_denominator","rating_numerator"],axis=1,inplace=True)

### df_csv ###
3- changing the dummy variabls into int type int( not boolean to get easy the discretive data even its int between 1 and 0)


In [44]:
df_csv["doggo"]=(df_csv["doggo"]=="doggo").astype(int)

In [45]:
df_csv["floofer"]=(df_csv["floofer"]=="floofer").astype(int)
df_csv["pupper"]=(df_csv["pupper"]=="pupper").astype(int)
df_csv["puppo"]=(df_csv["puppo"]=="puppo").astype(int)


df_csv.info()

In [47]:
df_csv["none"]= (df_csv["doggo"]+df_csv["floofer"]+df_csv["pupper"]+df_csv["puppo"])==0

In [48]:
df_csv["none"]=df_csv["none"].astype(int)

df_csv.describe()

### df_img  ###
1- chaging id into string

In [50]:
df_img["tweet_id"]=df_img["tweet_id"].astype(str)

df_img.info()

df_img.describe()

### df_api  ###
1- chaging id into string

In [53]:
df_api["tweet_id"]=df_api["tweet_id"].astype(str)

In [54]:
df_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2339 entries, 0 to 2338
Data columns (total 3 columns):
favorite_count    2339 non-null int64
retweet_count     2339 non-null int64
tweet_id          2339 non-null object
dtypes: int64(2), object(1)
memory usage: 54.9+ KB


df_api.describe()

# Cleaning #
## Tidiness ##

 ## df_api ,  df_csv ## 
represent the same obeservations it would better to be merged together but we should be careful about the nullable and deference count between the raw

In [56]:
merged_1=df_csv.merge(df_api,how="inner",on="tweet_id")

merged_1.head()

merged_1.describe()

merged_1.info()

##    ## df_img ,  merged_1 ## 
 df_img represent the same obeservations it would better to be merged together
 but we should be careful about the nullable and deference count between the raw

In [60]:
merged_2=merged_1.merge(df_img,how="inner",on="tweet_id")

merged_2.info()

merged_2.describe()

merged_2.head()

# Saving Data #
## saving as csv file ##



In [64]:
merged_2.to_csv("twitter_archive_master.csv",index_label="tweetid")

# Analysing and Visulisation #
please open the second notebook <b>act_reprot.ipynb</b>